*`Modelo de aprendizaje automático`**: 

Una vez que toda la data es consumible por la API, está lista para consumir por los departamentos de Analytics y Machine Learning, y nuestro EDA nos permite entender bien los datos a los que tenemos acceso, es hora de entrenar nuestro modelo de machine learning para armar un **sistema de recomendación**. Para ello, te ofrecen dos propuestas de trabajo: En la primera, el modelo deberá tener una relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. Aquí el input es un juego y el output es una lista de juegos recomendados, para ello recomendamos aplicar la *similitud del coseno*. 
La otra propuesta para el sistema de recomendación debe aplicar el filtro user-item, esto es tomar un usuario, se encuentran usuarios similares y se recomiendan ítems que a esos usuarios similares les gustaron. En este caso el input es un usuario y el output es una lista de juegos que se le recomienda a ese usuario, en general se explican como “A usuarios que son similares a tí también les gustó…”. 
Deben crear al menos **uno** de los dos sistemas de recomendación (Si se atreven a tomar el desafío, para mostrar su capacidad al equipo, ¡pueden hacer ambos!). Tu líder pide que el modelo derive obligatoriamente en un GET/POST en la API símil al siguiente formato:

Si es un sistema de recomendación item-item:
+ def **recomendacion_juego( *`id de producto`* )**:
    Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

Si es un sistema de recomendación user-item:
+ def **recomendacion_usuario( *`id de usuario`* )**:
    Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [41]:
# Importando librerias
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import pyarrow as pa
import pyarrow.parquet as pq

In [42]:
df_games = pd.read_parquet('C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosFinales/dfgamesrecomendacion.parquet', engine='pyarrow')
df_games.head(2)

,item_id,app_name,title,genres,release_date,developer,price
0,761140,Lost Summoner Kitty,Lost Summoner Kitty,Simulation,2018,Kotoshiro,4.99
1,761140,Lost Summoner Kitty,Lost Summoner Kitty,Casual,2018,Kotoshiro,4.99


In [43]:
#inicio un archivo con las columnas que tengo que reemplazar nulos
archivo_columnas=["app_name","developer","genres",'title']

#genero un dataframe con las columnas y sus nulos reemplazados por "Sin Datos"
nulos= df_games[archivo_columnas].fillna("Sin Datos")

#elimino las colummas a reemplazar y concateno ambos dataframes 
games_2 = pd.concat([df_games.drop(archivo_columnas, axis=1), nulos], axis=1)

In [44]:
modelo_item= games_2[["item_id", "app_name", "genres"]]  #SOLO TOMO LAS VARIABLES DE MI INTERES.
# EL ITEM ID QUE ES EL ID DE PRODUCTO,  el app y genres

In [45]:
modelo_item_csv = "C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosFinales/Modeloaprendizaje"
modelo_item.to_csv(modelo_item_csv , index=False, encoding="utf-8")

In [46]:
# Paso 1: Contar la frecuencia de cada género
genre_counts = modelo_item['genres'].value_counts()

# Paso 2: Seleccionar los 20 géneros más frecuentes
top_20_genres = genre_counts.head(20).index

# Paso 3: Filtrar los datos para incluir solo estos 20 géneros
filtered_modelo_item = modelo_item[modelo_item['genres'].isin(top_20_genres)]

# Paso 4: Convertir estos géneros seleccionados en variables dummy
modelo_item_2 = pd.get_dummies(filtered_modelo_item, columns=['genres'], prefix='', prefix_sep='')
# Mostrar el resultado
print(modelo_item_2.head())

   item_id             app_name     2D  Action  Adventure  Atmospheric  \
0   761140  Lost Summoner Kitty  False   False      False        False   
1   761140  Lost Summoner Kitty  False   False      False        False   
2   761140  Lost Summoner Kitty  False    True      False        False   
3   761140  Lost Summoner Kitty  False   False      False        False   
4   761140  Lost Summoner Kitty  False   False      False        False   

   Casual  Difficult  Early Access  Fantasy  ...  Indie  Multiplayer  \
0   False      False         False    False  ...  False        False   
1    True      False         False    False  ...  False        False   
2   False      False         False    False  ...  False        False   
3   False      False         False    False  ...  False        False   
4   False      False         False    False  ...   True        False   

   Platformer  Puzzle    RPG  Simulation  Singleplayer  Sports  Story Rich  \
0       False   False  False        True    

In [13]:
modelo_item_2.head(3)

,item_id,app_name,2D,Action,Adventure,Atmospheric,Casual,Difficult,Early Access,Fantasy,...,Indie,Multiplayer,Platformer,Puzzle,RPG,Simulation,Singleplayer,Sports,Story Rich,Strategy
0,761140,Lost Summoner Kitty,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,761140,Lost Summoner Kitty,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,761140,Lost Summoner Kitty,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [47]:
modelo_item_3= modelo_item_2.groupby(["item_id","app_name"]).sum().reset_index()
modelo_item_3.head(4)

,item_id,app_name,2D,Action,Adventure,Atmospheric,Casual,Difficult,Early Access,Fantasy,...,Indie,Multiplayer,Platformer,Puzzle,RPG,Simulation,Singleplayer,Sports,Story Rich,Strategy
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,20,Team Fortress Classic,0,1,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [50]:
modelo_item_3['item_id']=modelo_item_3['item_id'].astype('string')


In [51]:
modelo_item_3['item_id']=modelo_item_3['item_id'].astype('int')

In [52]:
# verificar todas las variables
modelo_item_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26755 entries, 0 to 26754
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           26755 non-null  int32 
 1   app_name          26755 non-null  object
 2   2D                26755 non-null  int64 
 3   Action            26755 non-null  int64 
 4   Adventure         26755 non-null  int64 
 5   Atmospheric       26755 non-null  int64 
 6   Casual            26755 non-null  int64 
 7   Difficult         26755 non-null  int64 
 8   Early Access      26755 non-null  int64 
 9   Fantasy           26755 non-null  int64 
 10  Free to Play      26755 non-null  int64 
 11  Great Soundtrack  26755 non-null  int64 
 12  Indie             26755 non-null  int64 
 13  Multiplayer       26755 non-null  int64 
 14  Platformer        26755 non-null  int64 
 15  Puzzle            26755 non-null  int64 
 16  RPG               26755 non-null  int64 
 17  Simulation  

In [53]:
# y aqui aplicamos el modelos que nos recomendaron, que es el modelo coseno
similitudes = cosine_similarity(modelo_item_3.iloc[:,3:])

In [54]:
similitudes

array([[1.        , 0.51639778, 0.66666667, ..., 0.81649658, 0.40824829,
        0.57735027],
       [0.51639778, 1.        , 0.51639778, ..., 0.63245553, 0.63245553,
        0.4472136 ],
       [0.66666667, 0.51639778, 1.        , ..., 0.81649658, 0.40824829,
        0.57735027],
       ...,
       [0.81649658, 0.63245553, 0.81649658, ..., 1.        , 0.5       ,
        0.70710678],
       [0.40824829, 0.63245553, 0.40824829, ..., 0.5       , 1.        ,
        0.70710678],
       [0.57735027, 0.4472136 , 0.57735027, ..., 0.70710678, 0.70710678,
        1.        ]])

In [55]:
# como tardo mucho vamos a hacer mas pequeños los datos, ya que si no la API NO VA A FUNCIONAR
modelo_final_csv = "modelo_final.csv" # esta tabla excede la memoria de la api
modelo_item_3.to_csv(modelo_final_csv , index=False, encoding="utf-8")

In [56]:
cant_filas= len(modelo_item_3)      # hacemos mas pequeños los datos
#Calculo la mitad
mitad_filas= cant_filas // 20
#Selecciono la mitad superior
modelo_render= modelo_item_3.iloc[:mitad_filas]

In [57]:
modelo_render.shape
modelo_render.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_id           1337 non-null   int32 
 1   app_name          1337 non-null   object
 2   2D                1337 non-null   int64 
 3   Action            1337 non-null   int64 
 4   Adventure         1337 non-null   int64 
 5   Atmospheric       1337 non-null   int64 
 6   Casual            1337 non-null   int64 
 7   Difficult         1337 non-null   int64 
 8   Early Access      1337 non-null   int64 
 9   Fantasy           1337 non-null   int64 
 10  Free to Play      1337 non-null   int64 
 11  Great Soundtrack  1337 non-null   int64 
 12  Indie             1337 non-null   int64 
 13  Multiplayer       1337 non-null   int64 
 14  Platformer        1337 non-null   int64 
 15  Puzzle            1337 non-null   int64 
 16  RPG               1337 non-null   int64 
 17  Simulation    

In [58]:
similitudes_render = cosine_similarity(modelo_render.iloc[:,3:])

In [59]:
modelo_render.head()

,item_id,app_name,2D,Action,Adventure,Atmospheric,Casual,Difficult,Early Access,Fantasy,...,Indie,Multiplayer,Platformer,Puzzle,RPG,Simulation,Singleplayer,Sports,Story Rich,Strategy
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,20,Team Fortress Classic,0,1,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,1,1,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0


In [60]:
modelo_render.to_parquet("ArchivosFinales/modelo_render.parquet")

In [61]:

# Guardar la muestra en la ruta especificada API
ruta_salida = 'C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosAPI/modelo_render.parquet'
modelo_render.to_parquet(ruta_salida, engine='pyarrow')

print(f'Se ha guardado una muestra del 10% del dataset en {ruta_salida}')

Se ha guardado una muestra del 10% del dataset en C:/Users/Edgar/OneDrive/Escritorio/ProyectoFinal1HenryNuevo/ArchivosAPI/modelo_render.parquet


In [62]:
modelo_render

,item_id,app_name,2D,Action,Adventure,Atmospheric,Casual,Difficult,Early Access,Fantasy,...,Indie,Multiplayer,Platformer,Puzzle,RPG,Simulation,Singleplayer,Sports,Story Rich,Strategy
0,10,Counter-Strike,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,20,Team Fortress Classic,0,1,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,30,Day of Defeat,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
3,40,Deathmatch Classic,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,50,Half-Life: Opposing Force,0,1,1,1,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,56480,"Warhammer 40,000: Dawn of War II: Retribution ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1333,56481,"Warhammer 40,000: Dawn of War II - Retribution...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1334,56482,"Warhammer 40,000: Dawn of War II - Retribution...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1335,56483,"Warhammer 40,000: Dawn of War II - Retribution...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [63]:
def recomendacion_juego(item_id): #este si funciono tal cual
    
    game = modelo_render[modelo_render['item_id'] == item_id]
    
    if game.empty:
        return("El juego '{item_id}' no posee registros.")
    
    # Obtiene el índice del juego dado
    idx = game.index[0]

    # Toma una muestra aleatoria del DataFrame df_games
    sample_size = 1000  # Define el tamaño de la muestra (ajusta según sea necesario)
    df_sample = modelo_render.sample(n=sample_size, random_state=42)  # Ajusta la semilla aleatoria según sea necesario

    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_render.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = df_sample['app_name'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [64]:
# ver las recomendaciones segun el item_id ejemplo
recomendacion_juego(2360)   # el juego es 204 counter strike el cual es un shooter de accion y las recomendaciones son parecidas.

{'similar_games': ['HeXen: Beyond Heretic',
  'Dungeon Siege III',
  'Hunted: The Demon’s Forge™',
  'SpellForce 2 - Anniversary Edition',
  'Dungeon Siege II']}

In [65]:
recomendacion_juego(10)  # el juego 35140 es batman y te recomienda otros videojuegos sobre peliculas y accion 

{'similar_games': ['Counter-Strike',
  'Counter-Strike: Global Offensive',
  'Command & Conquer: Red Alert 3 - Uprising',
  'Supreme Commander',
  'Empires Mod']}